<a href="https://colab.research.google.com/github/jungeun919/Pytorch_study/blob/main/CIFAR10_resnet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
# 신경망 학습 resnet34
from torchvision.models import resnet34
import torch.optim as optim


model = resnet34(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)


model.train()
for epoch in range(2):
  # running_loss = 0.0
  for i, data in enumerate(trainloader):
    inputs, labels = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # running_loss += loss.item()
    if i % 100 == 0:
      print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, loss.item()))

print('Finished Training')

[1,     1] loss: 2.707
[1,   101] loss: 1.350
[1,   201] loss: 1.005
[1,   301] loss: 0.961
[1,   401] loss: 0.973
[2,     1] loss: 0.594
[2,   101] loss: 0.627
[2,   201] loss: 0.722
[2,   301] loss: 0.651
[2,   401] loss: 0.573
Finished Training


In [14]:
# 전체 데이터셋의 정확도
correct, total = 0, 0
with torch.no_grad():
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy 10000 test images: %d %%' % (100 * correct / total))


# 각 클래스의 정확도
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    c = (predicted == labels.squeeze())
    for i in range(4):
      label = labels[i]
      class_correct[label] += c[i].item()
      class_total[label] += 1


for i in range(10):
  print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy 10000 test images: 76 %
Accuracy of plane : 88 %
Accuracy of   car : 90 %
Accuracy of  bird : 68 %
Accuracy of   cat : 76 %
Accuracy of  deer : 69 %
Accuracy of   dog : 71 %
Accuracy of  frog : 88 %
Accuracy of horse : 84 %
Accuracy of  ship : 91 %
Accuracy of truck : 65 %
